https://www.visualcrossing.com/resources/documentation/weather-api/how-to-use-python-and-pandas-to-join-historical-weather-data-to-road-accident-data/

=

https://www.visualcrossing.com/resources/documentation/weather-api/weather-api-json-result-structure/

=

https://www.visualcrossing.com/resources/documentation/weather-data/weather-data-documentation/

In [ ]:
with open('msk.geojson') as json_file:
  data_msk = json.load(json_file)
with open('mo.geojson') as json_file:
    data_mo = json.load(json_file)

mo_csv = pd.DataFrame()

for i in range(len(data_msk['features'])):
  first = data_msk['features'][i]
  points = first['properties']['point']
  if ("Туман" in (str(first['properties']['weather']))) or ("Метель" in (str(first['properties']['weather']))) or 
      ("Дождь" in (str(first['properties']['weather']))) or ("Ураганный ветер" in (str(first['properties']['weather']))) or 
        ("Снегопад" in (str(first['properties']['weather']))):
    df_small = {'datetime':first['properties']['datetime'], 'address':first['properties']['address'], 'lat':points['lat'], 'long':points['long'], 'weather':first['properties']['weather'], 'dead_count':first['properties']['dead_count'], 'injured_count':first['properties']['injured_count'], 'severity':first['properties']['severity']}
    mo_csv=mo_csv.append(df_small, ignore_index=True)

for i in range(len(data_mo['features'])):
  first = data_mo['features'][i]
  points = first['properties']['point']
  if ("Туман" in (str(first['properties']['weather']))) or ("Метель" in (str(first['properties']['weather']))) or 
      ("Дождь" in (str(first['properties']['weather']))) or ("Ураганный ветер" in (str(first['properties']['weather']))) or 
        ("Снегопад" in (str(first['properties']['weather']))):
    df_small = {'datetime':first['properties']['datetime'], 'address':first['properties']['address'], 'lat':points['lat'], 'long':points['long'], 'weather':first['properties']['weather'], 'dead_count':first['properties']['dead_count'], 'injured_count':first['properties']['injured_count'], 'severity':first['properties']['severity']}
    mo_csv=mo_csv.append(df_small,ignore_index=True)

mo_csv = mo_csv.sort_values(by='datetime')
mo_csv.to_csv('msk_and_mo.csv', encoding='utf-8-sig')

In [ ]:
import pandas as pd
import numpy as np
import ssl
import math
import datetime
import urllib.request
import json
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
ssl._create_default_https_context = ssl._create_unverified_context

Открываем .geojson, которые скачали из проекта "КартаДТП"

In [ ]:
with open('msk.geojson') as json_file:
  data_msk = json.load(json_file)

In [ ]:
with open('mo.geojson') as json_file:
    data_mo = json.load(json_file)

Проверяем состояние .json файла

In [ ]:
data_mo['features'][0]

{'geometry': {'coordinates': [38.0443, 55.8988], 'type': 'Point'},
 'properties': {'address': 'А-103 Щелковское шоссе Москва - Щелково - автомобильная дорога А-107 Московское малое кольцо (основное направление), 32 км',
  'category': 'Опрокидывание',
  'datetime': '2017-06-26 06:10:00',
  'dead_count': 1,
  'id': 2138637,
  'injured_count': 0,
  'light': 'Светлое время суток',
  'nearby': [],
  'parent_region': 'Московская область',
  'participant_categories': ['Все участники'],
  'participants': [],
  'participants_count': 1,
  'point': {'lat': 55.8988, 'long': 38.0443},
  'region': 'Щелковский',
  'road_conditions': ['Пыльное'],
  'scheme': '610',
  'severity': 'С погибшими',
  'tags': ['Дорожно-транспортные происшествия'],
  'vehicles': [{'brand': 'HYUNDAI',
    'category': 'В-класс (малый) до 3,9 м',
    'color': 'Белый',
    'model': 'Solaris',
    'participants': [{'gender': 'Женский',
      'health_status': 'Скончался на месте ДТП до приезда скорой медицинской помощи',
      'ro

Сортируем данные и выбираем только те ДТП, при которые были неблагоприятные погодные условия в Москве и Московской области. Объединяем в один датасет.

In [ ]:
c=0
mo_csv = pd.DataFrame()
for i in range(len(data_msk['features'])):
  first = data_msk['features'][i]
  points = first['properties']['point']
  if ("Туман" in (str(first['properties']['weather']))) or ("Метель" in (str(first['properties']['weather']))) or ("Дождь" in (str(first['properties']['weather']))) or ("Ураганный ветер" in (str(first['properties']['weather']))) or ("Снегопад" in (str(first['properties']['weather']))):
    df_small = {'datetime':first['properties']['datetime'], 'address':first['properties']['address'], 'lat':points['lat'], 'long':points['long'], 'weather':first['properties']['weather'], 'dead_count':first['properties']['dead_count'], 'injured_count':first['properties']['injured_count'], 'severity':first['properties']['severity']}
    mo_csv=mo_csv.append(df_small, ignore_index=True)
    c = c + 1
print("Количество аварий всего за 6 лет в Москве: ",len(data_msk['features']))
print("Количество аварий всего за 6 лет под влиянием метеофакторов в Москве: ",c)
print("Процент аварий под влиянием метеофакторов в Москве = ",(c/len(data_msk['features']))*100,"%")
print("===")

c=0
for i in range(len(data_mo['features'])):
  first = data_mo['features'][i]
  points = first['properties']['point']
  if ("Туман" in (str(first['properties']['weather']))) or ("Метель" in (str(first['properties']['weather']))) or ("Дождь" in (str(first['properties']['weather']))) or ("Ураганный ветер" in (str(first['properties']['weather']))) or ("Снегопад" in (str(first['properties']['weather']))):
    df_small = {'datetime':first['properties']['datetime'], 'address':first['properties']['address'], 'lat':points['lat'], 'long':points['long'], 'weather':first['properties']['weather'], 'dead_count':first['properties']['dead_count'], 'injured_count':first['properties']['injured_count'], 'severity':first['properties']['severity']}
    mo_csv=mo_csv.append(df_small,ignore_index=True)
    c = c + 1
print("Количество аварий всего за 6 лет в Московской области: ",len(data_mo['features']))
print("Количество аварий всего за 6 лет под влиянием метеофакторов в Московской области: ",c)
print("Процент аварий под влиянием метеофакторов в Московской области = ",(c/len(data_mo['features']))*100,"%")
print("===")

mo_csv = mo_csv.sort_values(by='datetime')
mo_csv.to_csv('msk_and_mo.csv', encoding='utf-8-sig')
print("Количество аварий всего за 6 лет под влиянием метеофакторов в Москве и МО: ", mo_csv.shape[0])

Другое первое значение в датафрейме, нежели когда записываем в новый датафрейм из файла.

In [ ]:
mo_csv['datetime'] = pd.to_datetime(mo_csv['datetime'])
mo_csv['lat'][0]

nan

In [ ]:
try:
  mo_csv = pd.read_csv('msk_and_mo.csv')
  mo_csv_with_weather = pd.read_csv('msk_and_mo_with_weather.csv')
except UnicodeDecodeError:
  print("bad")
mo_csv['datetime'] = pd.to_datetime(mo_csv['datetime'])
mo_csv.head(10)

,Unnamed: 0,datetime,address,lat,long,weather,dead_count,injured_count,severity
0,7050,2015-01-01 00:40:00,"д Кобяково (гп Голицыно), А-107 Московское мал...",55.542778,37.004167,"['Пасмурно', 'Снегопад']",0.0,1.0,Тяжёлый
1,7850,2015-01-01 01:00:00,"г Красногорск, ш Волоколамское",55.493056,37.184444,['Снегопад'],0.0,1.0,Тяжёлый
2,7550,2015-01-01 01:55:00,"д Бородино, ул Осташковская, 18-1",55.918889,37.661667,['Снегопад'],0.0,1.0,Легкий
3,607,2015-01-01 03:55:00,"г Москва, км МКАД 53-й",55.422400,37.233300,['Снегопад'],0.0,2.0,Тяжёлый
4,7019,2015-01-01 07:50:00,"г Одинцово, тер Можайское шоссе 27 км, 3-2",55.667778,37.255278,['Снегопад'],0.0,1.0,Легкий
5,4157,2015-01-01 17:20:00,"г Москва, ул Центральная",55.608056,37.352500,"['Пасмурно', 'Снегопад']",1.0,0.0,С погибшими
6,1098,2015-01-01 21:00:00,"г Москва, проезд Карамзина, 17-8",55.589444,37.541667,['Дождь'],0.0,1.0,Тяжёлый
7,7551,2015-01-01 22:00:00,"А-104 Москва - Дмитров – Дубна, 26 км",55.993333,37.533889,['Дождь'],0.0,1.0,Тяжёлый
8,3032,2015-01-02 05:40:00,"г Москва, пл Старая, 2-14",55.756389,37.630000,['Снегопад'],0.0,1.0,Легкий
9,2240,2015-01-02 18:27:00,"г Москва, ул Верхние Поля, 48А",55.656667,37.818889,"['Пасмурно', 'Дождь']",1.0,0.0,С погибшими


In [ ]:
mo_csv.columns
mo_csv_with_weather.shape[0]

8505

парснг одной аварии

In [ ]:
date = mo_csv['datetime'][20].date()
query_params = '&contentType=json&aggregateMinutes=15&unitGroup=us&includeAstronomy=true&collectStationContributions=true&key={}&startDateTime={}&endDateTime={}&locations={},{}'
query_params=query_params.format('DE2Q9MGYKB3EEPLKNCG92QSR3', date, date, mo_csv['lat'][20],mo_csv['long'][20])
response = urllib.request.urlopen('https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history'+'?'+ query_params)
data = response.read()
weatherData = json.loads(data.decode('utf-8'))
weather = weatherData['locations'][f"{mo_csv['lat'][20]},{mo_csv['long'][20]}"]
weather['values'][0]

{'cloudcover': 98.9,
 'conditions': 'Overcast',
 'datetime': 1420243200000,
 'datetimeStr': '2015-01-03T00:00:00+03:00',
 'dew': 33.0,
 'heatindex': None,
 'humidity': 96.63,
 'info': None,
 'maxt': 33.8,
 'mint': 33.8,
 'moonphase': 0.48,
 'precip': 0.0,
 'precipcover': None,
 'sealevelpressure': 988.0,
 'snow': None,
 'snowdepth': None,
 'solarenergy': None,
 'solarradiation': None,
 'stationinfo': 'MOSKVA VDNH DOLGOPRUDNYJ, RS (Id=27612099999,12.0mi), UUEE (Id=UUEE,22.0mi), UUWW (Id=UUWW,29.0mi), PAVLOVSKIJ POSAD, RS (Id=27523099999,30.0mi), UUDD (Id=UUDD,30.0mi), NOVO JERUSALIM, RS (Id=27511099999,44.0mi)',
 'sunrise': '2015-01-03T08:57:56+03:00',
 'sunset': '2015-01-03T16:07:30+03:00',
 'temp': 33.8,
 'visibility': 4.0,
 'wdir': 215.0,
 'weathertype': 'Snow Showers, Snow',
 'wgust': 22.4,
 'windchill': 26.5,
 'wspd': 8.9}

Парсим погодные данные в наш ранее созданный датасет и создаем новый, где уже включены ДТП и значение погодных условий.

In [ ]:
msk_mo = pd.DataFrame()
msk_mo=msk_mo.append(mo_csv_with_weather, ignore_index=True)
    
for i in range(8510,9015):
  date = mo_csv['datetime'][i].date()
  query_params = '&contentType=json&aggregateMinutes=15&unitGroup=us&includeAstronomy=true&collectStationContributions=true&key={}&startDateTime={}&endDateTime={}&locations={},{}'
  query_params=query_params.format('DE2Q9MGYKB3EEPLKNCG92QSR3', date, date, mo_csv['lat'][i],mo_csv['long'][i])
  response = urllib.request.urlopen('https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history'+'?'+ query_params)
  data = response.read()
  weatherData = json.loads(data.decode('utf-8'))
  try:
    weather = weatherData['locations'][f"{mo_csv['lat'][i]},{mo_csv['long'][i]}"]
    print(weather['values'][0]['visibility'],"->",i)
    df_small = {'datetime':mo_csv['datetime'][i], 'address':mo_csv['address'][i], 'lat':mo_csv['lat'][i], 'long':mo_csv['long'][i], 'weather':mo_csv['weather'][i], 'dead_count':mo_csv['dead_count'][i], 'injured_count':mo_csv['injured_count'][i], 'severity':mo_csv['severity'][i], 'cloudcover': weather['values'][0]['cloudcover'], 'dew': weather['values'][0]['dew'], 'humidity': weather['values'][0]['humidity'], 'precip': weather['values'][0]['precip'], 'snow': weather['values'][0]['snow'], 'snowdepth': weather['values'][0]['snowdepth'], 'temp': weather['values'][0]['temp'], 'visibility': weather['values'][0]['visibility'], 'weathertype': weather['values'][0]['weathertype'], 'wspd': weather['values'][0]['wspd'], 'wgust': weather['values'][0]['wgust'] }
    msk_mo=msk_mo.append(df_small, ignore_index=True)
    print("все окей","->",i)
  except KeyError:
    print("нет данных","->",i)
msk_mo.drop(columns = ['Unnamed: 0'],axis = 1, inplace=True)
#msk_mo.to_csv('msk_and_mo_with_weather.csv', encoding='utf-8-sig')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Удалить файл

In [ ]:
msk_mo.to_csv('msk_and_mo_with_weather.csv', encoding='utf-8-sig')

ЗАКОНЧИЛИ С ДАТАСЕТОМ

In [ ]:
import pandas as pd
import numpy as np
import ssl
import math
import datetime
import urllib.request
import json
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
msk_mo_with_weather = pd.read_csv('msk_and_mo_with_weather.csv')

Первые 50 строчек датафрейма, который в дальнейшем нужно обработать и можно использовать для обучения модели.

In [ ]:
msk_mo_with_weather.columns
msk_mo_with_weather.shape[0]

9010

In [ ]:
msk_mo_with_weather.head(2)

,Unnamed: 0,datetime,address,lat,long,weather,dead_count,injured_count,severity,cloudcover,dew,humidity,precip,snow,snowdepth,temp,visibility,weathertype,wspd,wgust
0,0,2015-01-01 00:40:00,"д Кобяково (гп Голицыно), А-107 Московское мал...",55.542778,37.004167,"['Пасмурно', 'Снегопад']",0.0,1.0,Тяжёлый,97.3,16.4,87.91,0.0,NaN,NaN,19.4,3.6,"Snow Showers, Drizzle, Sky Coverage Increasing",12.1,NaN
1,1,2015-01-01 01:00:00,"г Красногорск, ш Волоколамское",55.493056,37.184444,['Снегопад'],0.0,1.0,Тяжёлый,98.3,16.2,87.43,0.0,NaN,NaN,19.3,3.9,"Snow Showers, Drizzle, Sky Coverage Increasing",13.3,NaN


In [ ]:
#Легкий-1
#С погибшими-2
#Тяжёлый-3
severity = msk_mo_with_weather.groupby(by=['severity']).count()
severity

,Unnamed: 0,datetime,address,lat,long,weather,dead_count,injured_count,cloudcover,dew,humidity,precip,snow,snowdepth,temp,visibility,weathertype,wspd,wgust
severity,,,,,,,,,,,,,,,,,,,
Легкий,4419,4419,4285,4418,4418,4419,4419,4419,4277,4335,4335,4419,0,0,4335,4332,2425,4333,612
С погибшими,904,904,868,902,902,904,904,904,844,857,857,904,0,0,857,851,463,856,135
Тяжёлый,3687,3687,3552,3679,3679,3687,3687,3687,3468,3535,3535,3687,0,0,3535,3530,1961,3535,539


In [ ]:
'''
['Дождь'] = 1                          4899
['Снегопад'] = 2                      2808
['Пасмурно', 'Дождь'] = 3            696
['Пасмурно', 'Снегопад'] = 4            378
['Туман'] = 5                            88
['Пасмурно', 'Туман'] = 6                45
['Метель'] = 7                           26
['Дождь', 'Снегопад'] = 8                24
['Снегопад', 'Метель'] = 9               16
['Ясно', 'Туман'] = 10                    16
['Дождь', 'Туман'] = 11                    4
['Дождь', 'Ураганный ветер'] = 12          3
['Пасмурно', 'Метель'] = 13                3
['Ясно', 'Дождь'] = 14                     3
['Пасмурно', 'Ураганный ветер'] = 15         1
'''
weather = msk_mo_with_weather.groupby(by=['weather']).count()
weather = weather.sort_values(by=['Unnamed: 0'], ascending=False)
weather['Unnamed: 0']

weather
['Дождь']                          4899
['Снегопад']                       2808
['Пасмурно', 'Дождь']               696
['Пасмурно', 'Снегопад']            378
['Туман']                            88
['Пасмурно', 'Туман']                45
['Метель']                           26
['Дождь', 'Снегопад']                24
['Снегопад', 'Метель']               16
['Ясно', 'Туман']                    16
['Дождь', 'Туман']                    4
['Дождь', 'Ураганный ветер']          3
['Пасмурно', 'Метель']                3
['Ясно', 'Дождь']                     3
['Пасмурно', 'Ураганный ветер']       1
Name: Unnamed: 0, dtype: int64

In [ ]:
for i in range(msk_mo_with_weather.shape[0]):
  if (msk_mo_with_weather['severity'][i]=="Легкий"):
    msk_mo_with_weather['severity'][i] = 1;
  elif (msk_mo_with_weather['severity'][i]=="С погибшими"):
    msk_mo_with_weather['severity'][i] = 2;
  elif (msk_mo_with_weather['severity'][i]=="Тяжёлый"):
    msk_mo_with_weather['severity'][i] = 3;
  
  if (msk_mo_with_weather['weather'][i]=="['Дождь']"):
    msk_mo_with_weather['weather'][i] = 1;
  elif (msk_mo_with_weather['weather'][i]=="['Снегопад']"):
    msk_mo_with_weather['weather'][i] = 2;
  elif (msk_mo_with_weather['weather'][i]=="['Пасмурно', 'Дождь']"):
    msk_mo_with_weather['weather'][i] = 3;
  elif (msk_mo_with_weather['weather'][i]=="['Пасмурно', 'Снегопад']"):
    msk_mo_with_weather['weather'][i] = 4;
  elif (msk_mo_with_weather['weather'][i]=="['Туман']"):
    msk_mo_with_weather['weather'][i] = 5;
  elif (msk_mo_with_weather['weather'][i]=="['Пасмурно', 'Туман']"):
    msk_mo_with_weather['weather'][i] = 6;
  elif (msk_mo_with_weather['weather'][i]=="['Метель']"):
    msk_mo_with_weather['weather'][i] = 7;
  elif (msk_mo_with_weather['weather'][i]=="['Дождь', 'Снегопад']"):
    msk_mo_with_weather['weather'][i] = 8;
  elif (msk_mo_with_weather['weather'][i]=="['Снегопад', 'Метель']"):
    msk_mo_with_weather['weather'][i] = 9;
  elif (msk_mo_with_weather['weather'][i]=="['Ясно', 'Туман']"):
    msk_mo_with_weather['weather'][i] = 10;
  elif (msk_mo_with_weather['weather'][i]=="['Дождь', 'Туман']"):
    msk_mo_with_weather['weather'][i] = 11;
  elif (msk_mo_with_weather['weather'][i]=="['Дождь', 'Ураганный ветер']"):
    msk_mo_with_weather['weather'][i] = 12;
  elif (msk_mo_with_weather['weather'][i]=="['Пасмурно', 'Метель']"):
    msk_mo_with_weather['weather'][i] = 13;
  elif (msk_mo_with_weather['weather'][i]=="['Ясно', 'Дождь']"):
    msk_mo_with_weather['weather'][i] = 14;
  elif (msk_mo_with_weather['weather'][i]=="['Пасмурно', 'Ураганный ветер']"):
    msk_mo_with_weather['weather'][i] = 15;
    
msk_mo_with_weather.head(2)

Дропаем snow и snodeth, потому что они все пустые

In [ ]:
msk_mo_with_weather.drop(columns = ['snow'],axis = 1, inplace=True)
msk_mo_with_weather.drop(columns = ['snowdepth'],axis = 1, inplace=True)
msk_mo_with_weather.drop(columns = ['weathertype'],axis = 1, inplace=True)
msk_mo_with_weather.drop(columns = ['wgust'],axis = 1, inplace=True)

In [ ]:
weather = msk_mo_with_weather.groupby(by=['cloudcover']).count().sum()
weather['Unnamed: 0']

8589

In [ ]:
msk_mo_with_weather['cloudcover'].replace('', np.nan, inplace=True)
msk_mo_with_weather.dropna(subset=['cloudcover'], inplace=True)
msk_mo_with_weather

In [ ]:
msk_mo_with_weather['dew'].replace('', np.nan, inplace=True)
msk_mo_with_weather.dropna(subset=['dew'], inplace=True)
msk_mo_with_weather['humidity'].replace('', np.nan, inplace=True)
msk_mo_with_weather.dropna(subset=['humidity'], inplace=True)
msk_mo_with_weather['temp'].replace('', np.nan, inplace=True)
msk_mo_with_weather.dropna(subset=['temp'], inplace=True)
msk_mo_with_weather['visibility'].replace('', np.nan, inplace=True)
msk_mo_with_weather.dropna(subset=['visibility'], inplace=True)
msk_mo_with_weather['wspd'].replace('', np.nan, inplace=True)
msk_mo_with_weather.dropna(subset=['wspd'], inplace=True)
msk_mo_with_weather

In [ ]:
msk_mo_with_weather.to_csv('train.csv', encoding='utf-8-sig')

ФОРМИРОВАНИЯ ДАННЫХ ДЛЯ ОБУЧЕНИЯ

In [ ]:
msk_mo_with_weather.columns

Index(['Unnamed: 0', 'datetime', 'address', 'lat', 'long', 'weather',
       'dead_count', 'injured_count', 'severity', 'cloudcover', 'dew',
       'humidity', 'precip', 'temp', 'visibility', 'wspd'],
      dtype='object')

In [ ]:
x = msk_mo_with_weather[['cloudcover', 'dew', 'humidity', 'temp', 'visibility', 'wspd']]
y = msk_mo_with_weather['injured_count']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

ОБУЧАЕМ

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)

LinearRegression()

In [ ]:
m = RandomForestRegressor(n_estimators=90)
m.fit(x_train, y_train)

RandomForestRegressor(n_estimators=90)

Предсказываем

In [ ]:
predict = lr.predict(x_test)

In [ ]:
y_test.values[0]

1.0

In [ ]:
go = 0
for i in range(len(predict)):
  predict[i] = math.trunc(predict[i])
  if (y_test.values[i]==math.trunc(predict[i])):
    go=go+1;
  if (predict[i]==0):
    print(i)
print(go)

1306


In [ ]:
go/1714

0.7619603267211202

In [ ]:
#pd.set_option('display.max_rows', None)
pd.DataFrame([y_test.values, predict]).T

,0,1
0,1.0,1.0
1,2.0,1.0
2,1.0,1.0
3,2.0,1.0
4,0.0,1.0
...,...,...
1710,1.0,1.0
1711,14.0,1.0
1712,1.0,1.0
1713,1.0,1.0


In [ ]:
mean_squared_error(y_test, predict)

0.8262390670553936

In [ ]:
m_predict = m.predict(x_test)

In [ ]:
go = 0
for i in range(len(m_predict)):
  m_predict[i] = math.trunc(m_predict[i])
  if (y_test.values[i]==math.trunc(m_predict[i])):
    go=go+1;
  if (m_predict[i]==4):
    print(i)
print(go)

478
1094


In [ ]:
pd.set_option('display.max_rows', None)
pd.DataFrame([y_test.values, m_predict]).T

In [ ]:
go/1715

0.6618075801749271

In [ ]:
mean_squared_error(y_test, m_predict)

1.0192419825072887